📦 Cellule 1 : Imports et Configuration


In [13]:
# ========================================
# IMPORTS
# ========================================
import pandas as pd
import pyodbc
from dotenv import load_dotenv
import os
from tqdm import tqdm
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

print("=" * 70)
print("📊 ETL - CHARGEMENT DE FAIT_ENTREPRISE_AUTOMOBILE")
print("=" * 70)

# ========================================
# CHARGEMENT DES VARIABLES D'ENVIRONNEMENT
# ========================================
load_dotenv()

# Configuration DWH
DB_SERVER_DWH = os.getenv('DB_SERVER_DWH-student')
DB_DATABASE_DWH = os.getenv('DB_DATABASE_DWH')
DB_USERNAME_DWH = os.getenv('DB_USERNAME_DWH')
DB_PASSWORD_DWH = os.getenv('DB_PASSWORD_DWH')

print("\n✅ Variables d'environnement chargées")
print(f"   • DWH Server : {DB_SERVER_DWH}")
print(f"   • DWH Database : {DB_DATABASE_DWH}")

📊 ETL - CHARGEMENT DE FAIT_ENTREPRISE_AUTOMOBILE

✅ Variables d'environnement chargées
   • DWH Server : carter-cash-serveur-student.database.windows.net
   • DWH Database : DWH_E5_projet_AUTO


📦 Cellule 2 : Connexion au DWH


In [14]:
print("\n" + "=" * 70)
print("🔗 CONNEXION AU DATA WAREHOUSE")
print("=" * 70)

try:
    driver = '{ODBC Driver 17 for SQL Server}'
    
    connection_string_dwh = (
        'DRIVER=' + driver + 
        ';SERVER=' + DB_SERVER_DWH + 
        ';PORT=1433;DATABASE=' + DB_DATABASE_DWH + 
        ';UID=' + DB_USERNAME_DWH + 
        ';PWD=' + DB_PASSWORD_DWH
    )
    
    cnxn_dwh = pyodbc.connect(connection_string_dwh)
    cursor_dwh = cnxn_dwh.cursor()
    
    print(f"✅ Connexion réussie au DWH")
    print(f"   • Serveur : {DB_SERVER_DWH}")
    print(f"   • Base de données : {DB_DATABASE_DWH}")
    
except Exception as e:
    print(f"❌ Erreur de connexion au DWH : {e}")
    raise


🔗 CONNEXION AU DATA WAREHOUSE
✅ Connexion réussie au DWH
   • Serveur : carter-cash-serveur-student.database.windows.net
   • Base de données : DWH_E5_projet_AUTO


📦 Cellule 3 : Initialisation du Rapport de Tests


In [15]:
print("\n" + "=" * 70)
print("📋 INITIALISATION DU RAPPORT DE TESTS")
print("=" * 70)

# ========================================
# STRUCTURE DU RAPPORT
# ========================================
rapport_tests = []

def ajouter_resultat_test(numero, nom_test, table_cible, objectif, resultat_valeur, seuil_attendu, statut, commentaire=""):
    """Ajoute un résultat de test au rapport"""
    rapport_tests.append({
        'N°': numero,
        'Test': nom_test,
        'Table': table_cible,
        'Objectif': objectif,
        'Résultat': resultat_valeur,
        'Seuil': seuil_attendu,
        'Statut': statut,
        'Commentaire': commentaire
    })

print("✅ Système de rapport initialisé")
print(f"   • Format : DataFrame pandas")
print(f"   • Export prévu : CSV + Console")


📋 INITIALISATION DU RAPPORT DE TESTS
✅ Système de rapport initialisé
   • Format : DataFrame pandas
   • Export prévu : CSV + Console


📦 Cellule 4 : TEST 1 - Nullité des Clés Techniques (PK/FK)


In [16]:
print("\n" + "=" * 70)
print("🔍 TEST 1 : NULLITÉ DES CLÉS TECHNIQUES (PK/FK)")
print("=" * 70)

# ========================================
# LISTE DES TABLES ET CLÉS À TESTER
# ========================================
tables_dimensions = [
    ('DIM_TEMPS', 'SK_Temps'),
    ('DIM_GEOGRAPHIE', 'SK_Geographie'),
    ('DIM_ACTIVITE_NAF', 'SK_Activite'),
    ('DIM_ENTREPRISE', 'SK_Entreprise'),
    ('DIM_BORNE_RECHARGE', 'SK_Borne'),
    ('DIM_USER_API', 'SK_User_API'),
    ('DIM_VEHICULE', 'SK_Vehicule'),
    ('DIM_PRODUIT_PNEU', 'SK_Produit'),
    ('DIM_CARACTERISTIQUES_PNEU', 'SK_Caracteristique'),
    ('DIM_DIMENSIONS_PNEU', 'SK_Dimension_Pneu')
]

tables_faits_fk = [
    ('FAIT_PRIX_PNEU', ['SK_Temps', 'SK_Produit', 'SK_Caracteristique', 'SK_Dimension_Pneu']),
    ('FAIT_DISPONIBILITE_BORNE', ['SK_Temps', 'SK_Borne', 'SK_Geographie']),
    ('FAIT_ENTREPRISE_AUTOMOBILE', ['SK_Temps', 'SK_Entreprise', 'SK_Activite', 'SK_Geographie'])
]

print("\n⏳ Test des clés primaires dans les tables de DIMENSIONS...")

test_numero = 1
erreurs_pk = []

for table, pk in tables_dimensions:
    query = f"SELECT COUNT(*) FROM {table} WHERE {pk} IS NULL"
    try:
        cursor_dwh.execute(query)
        nb_null = cursor_dwh.fetchone()[0]
        
        statut = "✅ OK" if nb_null == 0 else "❌ KO"
        commentaire = "" if nb_null == 0 else f"{nb_null} clés NULL détectées"
        
        ajouter_resultat_test(
            test_numero,
            f"Nullité PK {pk}",
            table,
            "PK ne doit jamais être NULL",
            nb_null,
            0,
            statut,
            commentaire
        )
        
        if nb_null > 0:
            erreurs_pk.append(f"{table}.{pk}")
        
        print(f"   {statut} {table}.{pk} : {nb_null} NULL")
        test_numero += 1
        
    except Exception as e:
        print(f"   ❌ ERREUR {table}.{pk} : {e}")
        ajouter_resultat_test(
            test_numero,
            f"Nullité PK {pk}",
            table,
            "PK ne doit jamais être NULL",
            "ERREUR",
            0,
            "❌ ERREUR",
            str(e)
        )
        test_numero += 1

print("\n⏳ Test des clés étrangères dans les tables de FAITS...")

for table, fks in tables_faits_fk:
    for fk in fks:
        query = f"SELECT COUNT(*) FROM {table} WHERE {fk} IS NULL"
        try:
            cursor_dwh.execute(query)
            nb_null = cursor_dwh.fetchone()[0]
            
            statut = "✅ OK" if nb_null == 0 else "❌ KO"
            commentaire = "" if nb_null == 0 else f"{nb_null} FK NULL détectées"
            
            ajouter_resultat_test(
                test_numero,
                f"Nullité FK {fk}",
                table,
                "FK ne doit jamais être NULL",
                nb_null,
                0,
                statut,
                commentaire
            )
            
            if nb_null > 0:
                erreurs_pk.append(f"{table}.{fk}")
            
            print(f"   {statut} {table}.{fk} : {nb_null} NULL")
            test_numero += 1
            
        except Exception as e:
            print(f"   ❌ ERREUR {table}.{fk} : {e}")
            ajouter_resultat_test(
                test_numero,
                f"Nullité FK {fk}",
                table,
                "FK ne doit jamais être NULL",
                "ERREUR",
                0,
                "❌ ERREUR",
                str(e)
            )
            test_numero += 1

if len(erreurs_pk) == 0:
    print("\n✅ TEST 1 RÉUSSI : Aucune clé NULL détectée")
else:
    print(f"\n⚠️  TEST 1 PARTIELLEMENT ÉCHOUÉ : {len(erreurs_pk)} anomalies détectées")
    print(f"   Tables concernées : {', '.join(erreurs_pk)}")


🔍 TEST 1 : NULLITÉ DES CLÉS TECHNIQUES (PK/FK)

⏳ Test des clés primaires dans les tables de DIMENSIONS...
   ✅ OK DIM_TEMPS.SK_Temps : 0 NULL
   ✅ OK DIM_GEOGRAPHIE.SK_Geographie : 0 NULL
   ✅ OK DIM_ACTIVITE_NAF.SK_Activite : 0 NULL
   ✅ OK DIM_ENTREPRISE.SK_Entreprise : 0 NULL
   ✅ OK DIM_BORNE_RECHARGE.SK_Borne : 0 NULL
   ✅ OK DIM_USER_API.SK_User_API : 0 NULL
   ✅ OK DIM_VEHICULE.SK_Vehicule : 0 NULL
   ✅ OK DIM_PRODUIT_PNEU.SK_Produit : 0 NULL
   ✅ OK DIM_CARACTERISTIQUES_PNEU.SK_Caracteristique : 0 NULL
   ✅ OK DIM_DIMENSIONS_PNEU.SK_Dimension_Pneu : 0 NULL

⏳ Test des clés étrangères dans les tables de FAITS...
   ✅ OK FAIT_PRIX_PNEU.SK_Temps : 0 NULL
   ✅ OK FAIT_PRIX_PNEU.SK_Produit : 0 NULL
   ✅ OK FAIT_PRIX_PNEU.SK_Caracteristique : 0 NULL
   ✅ OK FAIT_PRIX_PNEU.SK_Dimension_Pneu : 0 NULL
   ✅ OK FAIT_DISPONIBILITE_BORNE.SK_Temps : 0 NULL
   ✅ OK FAIT_DISPONIBILITE_BORNE.SK_Borne : 0 NULL
   ✅ OK FAIT_DISPONIBILITE_BORNE.SK_Geographie : 0 NULL
   ✅ OK FAIT_ENTREPRISE_AUTO

📦 Cellule 5 : TEST 2 - Unicité des Clés Primaires


In [17]:
print("\n" + "=" * 70)
print("🔍 TEST 2 : UNICITÉ DES CLÉS PRIMAIRES")
print("=" * 70)

print("\n⏳ Vérification de l'unicité des PK dans les tables de DIMENSIONS...")

erreurs_unicite = []

for table, pk in tables_dimensions:
    query = f"""
    SELECT 
        COUNT(*) as Total_Lignes,
        COUNT(DISTINCT {pk}) as PK_Uniques
    FROM {table}
    """
    
    try:
        cursor_dwh.execute(query)
        row = cursor_dwh.fetchone()
        total_lignes = row[0]
        pk_uniques = row[1]
        
        doublons = total_lignes - pk_uniques
        
        statut = "✅ OK" if doublons == 0 else "❌ KO"
        commentaire = "" if doublons == 0 else f"{doublons} doublons détectés"
        
        ajouter_resultat_test(
            test_numero,
            f"Unicité PK {pk}",
            table,
            "Nombre lignes = Nombre PK distinctes",
            f"{total_lignes} lignes, {pk_uniques} uniques",
            "Égalité",
            statut,
            commentaire
        )
        
        if doublons > 0:
            erreurs_unicite.append(f"{table}.{pk}")
        
        print(f"   {statut} {table} : {total_lignes} lignes, {pk_uniques} PK uniques (diff: {doublons})")
        test_numero += 1
        
    except Exception as e:
        print(f"   ❌ ERREUR {table}.{pk} : {e}")
        ajouter_resultat_test(
            test_numero,
            f"Unicité PK {pk}",
            table,
            "Nombre lignes = Nombre PK distinctes",
            "ERREUR",
            "Égalité",
            "❌ ERREUR",
            str(e)
        )
        test_numero += 1

if len(erreurs_unicite) == 0:
    print("\n✅ TEST 2 RÉUSSI : Toutes les PK sont uniques")
else:
    print(f"\n⚠️  TEST 2 ÉCHOUÉ : {len(erreurs_unicite)} tables avec doublons")
    print(f"   Tables concernées : {', '.join(erreurs_unicite)}")


🔍 TEST 2 : UNICITÉ DES CLÉS PRIMAIRES

⏳ Vérification de l'unicité des PK dans les tables de DIMENSIONS...
   ✅ OK DIM_TEMPS : 4018 lignes, 4018 PK uniques (diff: 0)
   ✅ OK DIM_GEOGRAPHIE : 3788 lignes, 3788 PK uniques (diff: 0)
   ✅ OK DIM_ACTIVITE_NAF : 18 lignes, 18 PK uniques (diff: 0)
   ✅ OK DIM_ENTREPRISE : 19028 lignes, 19028 PK uniques (diff: 0)
   ✅ OK DIM_BORNE_RECHARGE : 913 lignes, 913 PK uniques (diff: 0)
   ✅ OK DIM_USER_API : 4 lignes, 4 PK uniques (diff: 0)
   ✅ OK DIM_VEHICULE : 63322 lignes, 63322 PK uniques (diff: 0)
   ✅ OK DIM_PRODUIT_PNEU : 11442 lignes, 11442 PK uniques (diff: 0)
   ✅ OK DIM_CARACTERISTIQUES_PNEU : 687 lignes, 687 PK uniques (diff: 0)
   ✅ OK DIM_DIMENSIONS_PNEU : 747 lignes, 747 PK uniques (diff: 0)

✅ TEST 2 RÉUSSI : Toutes les PK sont uniques


📦 Cellule 6 : TEST 3 - Intégrité Référentielle


In [18]:
print("\n" + "=" * 70)
print("🔍 TEST 3 : INTÉGRITÉ RÉFÉRENTIELLE (FK → DIM)")
print("=" * 70)

# ========================================
# DÉFINITION DES RELATIONS FK → PK
# ========================================
relations_fk = [
    # FAIT_PRIX_PNEU
    ('FAIT_PRIX_PNEU', 'SK_Temps', 'DIM_TEMPS', 'SK_Temps'),
    ('FAIT_PRIX_PNEU', 'SK_Produit', 'DIM_PRODUIT_PNEU', 'SK_Produit'),
    ('FAIT_PRIX_PNEU', 'SK_Caracteristique', 'DIM_CARACTERISTIQUES_PNEU', 'SK_Caracteristique'),
    ('FAIT_PRIX_PNEU', 'SK_Dimension_Pneu', 'DIM_DIMENSIONS_PNEU', 'SK_Dimension_Pneu'),
    
    # FAIT_DISPONIBILITE_BORNE
    ('FAIT_DISPONIBILITE_BORNE', 'SK_Temps', 'DIM_TEMPS', 'SK_Temps'),
    ('FAIT_DISPONIBILITE_BORNE', 'SK_Borne', 'DIM_BORNE_RECHARGE', 'SK_Borne'),
    ('FAIT_DISPONIBILITE_BORNE', 'SK_Geographie', 'DIM_GEOGRAPHIE', 'SK_Geographie'),
    
    # FAIT_ENTREPRISE_AUTOMOBILE
    ('FAIT_ENTREPRISE_AUTOMOBILE', 'SK_Temps', 'DIM_TEMPS', 'SK_Temps'),
    ('FAIT_ENTREPRISE_AUTOMOBILE', 'SK_Entreprise', 'DIM_ENTREPRISE', 'SK_Entreprise'),
    ('FAIT_ENTREPRISE_AUTOMOBILE', 'SK_Activite', 'DIM_ACTIVITE_NAF', 'SK_Activite'),
    ('FAIT_ENTREPRISE_AUTOMOBILE', 'SK_Geographie', 'DIM_GEOGRAPHIE', 'SK_Geographie'),
]

print("\n⏳ Vérification des relations FK → PK...")

erreurs_integrite = []

for table_fait, fk, table_dim, pk in relations_fk:
    query = f"""
    SELECT COUNT(*)
    FROM {table_fait} f
    LEFT JOIN {table_dim} d ON f.{fk} = d.{pk}
    WHERE d.{pk} IS NULL
    """
    
    try:
        cursor_dwh.execute(query)
        nb_orphelins = cursor_dwh.fetchone()[0]
        
        statut = "✅ OK" if nb_orphelins == 0 else "❌ KO"
        commentaire = "" if nb_orphelins == 0 else f"{nb_orphelins} lignes orphelines"
        
        ajouter_resultat_test(
            test_numero,
            f"Intégrité FK",
            f"{table_fait} → {table_dim}",
            "Aucune ligne orpheline",
            nb_orphelins,
            0,
            statut,
            commentaire
        )
        
        if nb_orphelins > 0:
            erreurs_integrite.append(f"{table_fait}.{fk} → {table_dim}.{pk}")
        
        print(f"   {statut} {table_fait}.{fk} → {table_dim}.{pk} : {nb_orphelins} orphelins")
        test_numero += 1
        
    except Exception as e:
        print(f"   ❌ ERREUR {table_fait}.{fk} → {table_dim}.{pk} : {e}")
        ajouter_resultat_test(
            test_numero,
            f"Intégrité FK",
            f"{table_fait} → {table_dim}",
            "Aucune ligne orpheline",
            "ERREUR",
            0,
            "❌ ERREUR",
            str(e)
        )
        test_numero += 1

if len(erreurs_integrite) == 0:
    print("\n✅ TEST 3 RÉUSSI : Intégrité référentielle respectée")
else:
    print(f"\n⚠️  TEST 3 ÉCHOUÉ : {len(erreurs_integrite)} relations avec orphelins")
    for erreur in erreurs_integrite:
        print(f"   - {erreur}")


🔍 TEST 3 : INTÉGRITÉ RÉFÉRENTIELLE (FK → DIM)

⏳ Vérification des relations FK → PK...
   ✅ OK FAIT_PRIX_PNEU.SK_Temps → DIM_TEMPS.SK_Temps : 0 orphelins
   ✅ OK FAIT_PRIX_PNEU.SK_Produit → DIM_PRODUIT_PNEU.SK_Produit : 0 orphelins
   ✅ OK FAIT_PRIX_PNEU.SK_Caracteristique → DIM_CARACTERISTIQUES_PNEU.SK_Caracteristique : 0 orphelins
   ✅ OK FAIT_PRIX_PNEU.SK_Dimension_Pneu → DIM_DIMENSIONS_PNEU.SK_Dimension_Pneu : 0 orphelins
   ✅ OK FAIT_DISPONIBILITE_BORNE.SK_Temps → DIM_TEMPS.SK_Temps : 0 orphelins
   ✅ OK FAIT_DISPONIBILITE_BORNE.SK_Borne → DIM_BORNE_RECHARGE.SK_Borne : 0 orphelins
   ✅ OK FAIT_DISPONIBILITE_BORNE.SK_Geographie → DIM_GEOGRAPHIE.SK_Geographie : 0 orphelins
   ✅ OK FAIT_ENTREPRISE_AUTOMOBILE.SK_Temps → DIM_TEMPS.SK_Temps : 0 orphelins
   ✅ OK FAIT_ENTREPRISE_AUTOMOBILE.SK_Entreprise → DIM_ENTREPRISE.SK_Entreprise : 0 orphelins
   ✅ OK FAIT_ENTREPRISE_AUTOMOBILE.SK_Activite → DIM_ACTIVITE_NAF.SK_Activite : 0 orphelins
   ✅ OK FAIT_ENTREPRISE_AUTOMOBILE.SK_Geographie 

📦 Cellule 7 : TEST 4 - Doublons sur Clés Métier


In [19]:
print("\n" + "=" * 70)
print("🔍 TEST 4 : DOUBLONS SUR CLÉS MÉTIER")
print("=" * 70)

# ========================================
# TESTS DE DOUBLONS SUR CLÉS MÉTIER
# ========================================
tests_doublons = [
    ('DIM_ENTREPRISE', 'SIRET', 'Un SIRET doit être unique'),
    ('DIM_ACTIVITE_NAF', 'Code_NAF', 'Un code NAF doit être unique'),
    ('DIM_GEOGRAPHIE', 'Code_INSEE', 'Un code INSEE doit être unique'),
    ('DIM_BORNE_RECHARGE', 'ID_Station', 'Un ID station doit être unique'),
]

print("\n⏳ Vérification des doublons sur clés métier...")

erreurs_doublons = []

for table, cle_metier, description in tests_doublons:
    query = f"""
    SELECT 
        {cle_metier},
        COUNT(*) as Nb_Occurrences
    FROM {table}
    WHERE {cle_metier} IS NOT NULL
    GROUP BY {cle_metier}
    HAVING COUNT(*) > 1
    """
    
    try:
        cursor_dwh.execute(query)
        doublons = cursor_dwh.fetchall()
        nb_doublons = len(doublons)
        
        statut = "✅ OK" if nb_doublons == 0 else "❌ KO"
        commentaire = "" if nb_doublons == 0 else f"{nb_doublons} valeurs en doublon"
        
        ajouter_resultat_test(
            test_numero,
            f"Doublons {cle_metier}",
            table,
            description,
            nb_doublons,
            0,
            statut,
            commentaire
        )
        
        if nb_doublons > 0:
            erreurs_doublons.append(f"{table}.{cle_metier}")
            print(f"   ❌ KO {table}.{cle_metier} : {nb_doublons} valeurs en doublon")
            # Afficher quelques exemples
            for i, (valeur, count) in enumerate(doublons[:3]):
                print(f"      Exemple : {valeur} ({count} occurrences)")
        else:
            print(f"   ✅ OK {table}.{cle_metier} : Aucun doublon")
        
        test_numero += 1
        
    except Exception as e:
        print(f"   ❌ ERREUR {table}.{cle_metier} : {e}")
        ajouter_resultat_test(
            test_numero,
            f"Doublons {cle_metier}",
            table,
            description,
            "ERREUR",
            0,
            "❌ ERREUR",
            str(e)
        )
        test_numero += 1

if len(erreurs_doublons) == 0:
    print("\n✅ TEST 4 RÉUSSI : Aucun doublon sur clés métier")
else:
    print(f"\n⚠️  TEST 4 ÉCHOUÉ : {len(erreurs_doublons)} tables avec doublons")
    for erreur in erreurs_doublons:
        print(f"   - {erreur}")


🔍 TEST 4 : DOUBLONS SUR CLÉS MÉTIER

⏳ Vérification des doublons sur clés métier...
   ✅ OK DIM_ENTREPRISE.SIRET : Aucun doublon
   ✅ OK DIM_ACTIVITE_NAF.Code_NAF : Aucun doublon
   ✅ OK DIM_GEOGRAPHIE.Code_INSEE : Aucun doublon
   ❌ KO DIM_BORNE_RECHARGE.ID_Station : 119 valeurs en doublon
      Exemple : FR*E11*PLMTOURCOING59212*1 (2 occurrences)
      Exemple : FR*E45*PIMTDOUAI59508*1 (2 occurrences)
      Exemple : FR*E45*PIMTDOUAI59508*2 (2 occurrences)

⚠️  TEST 4 ÉCHOUÉ : 1 tables avec doublons
   - DIM_BORNE_RECHARGE.ID_Station


📦 Cellule 8 : TEST 5 - Format et Cohérence des Données


In [20]:
print("\n" + "=" * 70)
print("🔍 TEST 5 : FORMAT ET COHÉRENCE DES DONNÉES (EXEMPLES CONCRETS)")
print("=" * 70)

# Test 5.1 : Population > 0 dans DIM_GEOGRAPHIE
print("\n⏳ Test 5.1 : Communes avec population ≤ 0...")

query = """
SELECT TOP 5 SK_Geographie, Code_INSEE, Nom_Commune, Population, Superficie_km2
FROM DIM_GEOGRAPHIE
WHERE Population IS NOT NULL AND Population <= 0
ORDER BY Population
"""
try:
    df_pop = pd.read_sql(query, cnxn_dwh)
    print(df_pop)
    nb_anomalies = len(df_pop)
    if nb_anomalies > 0:
        print(f"\n   👉 Exemples affichés ci-dessus. Pour tout voir, utilisez la requête sans TOP 5.")
    else:
        print("   ✅ Aucun cas détecté.")
except Exception as e:
    print(f"   ❌ Erreur d'extraction exemples population : {e}")

# Test 5.2 : Superficie > 0 dans DIM_GEOGRAPHIE
print("\n⏳ Test 5.2 : Communes avec superficie ≤ 0...")

query = """
SELECT TOP 5 SK_Geographie, Code_INSEE, Nom_Commune, Population, Superficie_km2
FROM DIM_GEOGRAPHIE
WHERE Superficie_km2 IS NOT NULL AND Superficie_km2 <= 0
ORDER BY Superficie_km2
"""
try:
    df_sup = pd.read_sql(query, cnxn_dwh)
    print(df_sup)
    nb_anomalies = len(df_sup)
    if nb_anomalies > 0:
        print(f"\n   👉 Exemples affichés ci-dessus. Pour tout voir, utilisez la requête sans TOP 5.")
    else:
        print("   ✅ Aucun cas détecté.")
except Exception as e:
    print(f"   ❌ Erreur d'extraction exemples superficie : {e}")

# Test 5.3 : Code INSEE mal formatés (pas 5 caractères)
print("\n⏳ Test 5.3 : Codes INSEE mal formatés...")

query = """
SELECT TOP 5 SK_Geographie, Code_INSEE, Nom_Commune
FROM DIM_GEOGRAPHIE
WHERE Code_INSEE IS NOT NULL AND LEN(Code_INSEE) != 5
ORDER BY Code_INSEE
"""
try:
    df_insee = pd.read_sql(query, cnxn_dwh)
    print(df_insee)
    nb_anomalies = len(df_insee)
    if nb_anomalies > 0:
        print(f"\n   👉 Exemples affichés ci-dessus. Pour tout voir, utilisez la requête sans TOP 5.")
    else:
        print("   ✅ Aucun cas détecté.")
except Exception as e:
    print(f"   ❌ Erreur d'extraction exemples code INSEE : {e}")

# Test 5.4 : NAF non vide dans DIM_ENTREPRISE
print("\n⏳ Test 5.4 : Entreprises sans code NAF...")

query = """
SELECT TOP 5 SK_Entreprise, SIRET, Denomination, NAF
FROM DIM_ENTREPRISE
WHERE NAF IS NULL OR NAF = ''
ORDER BY SK_Entreprise
"""
try:
    df_naf = pd.read_sql(query, cnxn_dwh)
    print(df_naf)
    nb_anomalies = len(df_naf)
    if nb_anomalies > 0:
        print(f"\n   👉 Exemples affichés ci-dessus. Pour tout voir, utilisez la requête sans TOP 5.")
    else:
        print("   ✅ Aucun cas détecté.")
except Exception as e:
    print(f"   ❌ Erreur d'extraction exemples code NAF : {e}")

# Test 5.5 : Prix > 0 dans FAIT_PRIX_PNEU
print("\n⏳ Test 5.5 : Prix ≤ 0 dans FAIT_PRIX_PNEU...")

query = """
SELECT TOP 5 SK_Temps, SK_Produit, Prix_Euro, Date_Scrap
FROM FAIT_PRIX_PNEU
WHERE Prix_Euro IS NOT NULL AND Prix_Euro <= 0
ORDER BY Prix_Euro
"""
try:
    df_prix = pd.read_sql(query, cnxn_dwh)
    print(df_prix)
    nb_anomalies = len(df_prix)
    if nb_anomalies > 0:
        print(f"\n   👉 Exemples affichés ci-dessus. Pour tout voir, utilisez la requête sans TOP 5.")
    else:
        print("   ✅ Aucun cas détecté.")
except Exception as e:
    print(f"   ❌ Erreur d'extraction exemples prix pneu : {e}")

print("\n✅ TEST 5 AVEC EXTRAITS D'EXEMPLES TERMINÉ")


🔍 TEST 5 : FORMAT ET COHÉRENCE DES DONNÉES (EXEMPLES CONCRETS)

⏳ Test 5.1 : Communes avec population ≤ 0...
   SK_Geographie Code_INSEE        Nom_Commune  Population  Superficie_km2
0           2118      60694  Les Hauts-Talican           0             5.0

   👉 Exemples affichés ci-dessus. Pour tout voir, utilisez la requête sans TOP 5.

⏳ Test 5.2 : Communes avec superficie ≤ 0...
   SK_Geographie Code_INSEE Nom_Commune  Population  Superficie_km2
0           1122      59332      Lannoy        1800             0.0
1            139       2143  Le Catelet         185             0.0

   👉 Exemples affichés ci-dessus. Pour tout voir, utilisez la requête sans TOP 5.

⏳ Test 5.3 : Codes INSEE mal formatés...
   SK_Geographie Code_INSEE             Nom_Commune
0              1       2001               Abbécourt
1              2       2002                  Achery
2              3       2003                     Acy
3              4       2004  Agnicourt-et-Séchelles
4              5      

📦 Cellule 9 : TEST 6 - Volumétrie et Cohérence


In [21]:
print("\n" + "=" * 70)
print("🔍 TEST 6 : TESTS DE VOLUMÉTRIE ET COHÉRENCE")
print("=" * 70)

# ========================================
# COMPTAGE GLOBAL PAR TABLE
# ========================================
tables_a_compter = [
    'DIM_TEMPS',
    'DIM_GEOGRAPHIE',
    'DIM_ACTIVITE_NAF',
    'DIM_ENTREPRISE',
    'DIM_BORNE_RECHARGE',
    'DIM_USER_API',
    'DIM_VEHICULE',
    'DIM_PRODUIT_PNEU',
    'DIM_CARACTERISTIQUES_PNEU',
    'DIM_DIMENSIONS_PNEU',
    'FAIT_PRIX_PNEU',
    'FAIT_DISPONIBILITE_BORNE',
    'FAIT_ENTREPRISE_AUTOMOBILE'
]

print("\n⏳ Comptage du nombre d'enregistrements par table...")

volumetrie = {}

for table in tables_a_compter:
    query = f"SELECT COUNT(*) FROM {table}"
    try:
        cursor_dwh.execute(query)
        nb_lignes = cursor_dwh.fetchone()[0]
        volumetrie[table] = nb_lignes
        
        # Définir des seuils minimums attendus
        seuil_min = 1  # Au moins 1 ligne
        
        statut = "✅ OK" if nb_lignes >= seuil_min else "⚠️  WARNING"
        commentaire = "" if nb_lignes >= seuil_min else "Table vide ou presque vide"
        
        ajouter_resultat_test(
            test_numero,
            f"Volumétrie {table}",
            table,
            f"Au moins {seuil_min} enregistrement(s)",
            nb_lignes,
            seuil_min,
            statut,
            commentaire
        )
        
        print(f"   {statut} {table:<40} : {nb_lignes:>10} lignes")
        test_numero += 1
        
    except Exception as e:
        print(f"   ❌ ERREUR {table} : {e}")
        volumetrie[table] = "ERREUR"
        test_numero += 1

print("\n📊 SYNTHÈSE DE LA VOLUMÉTRIE :")
print(f"   • Total DIMENSIONS : {sum([v for k, v in volumetrie.items() if k.startswith('DIM_') and isinstance(v, int)])} lignes")
print(f"   • Total FAITS : {sum([v for k, v in volumetrie.items() if k.startswith('FAIT_') and isinstance(v, int)])} lignes")

print("\n✅ TEST 6 TERMINÉ")


🔍 TEST 6 : TESTS DE VOLUMÉTRIE ET COHÉRENCE

⏳ Comptage du nombre d'enregistrements par table...
   ✅ OK DIM_TEMPS                                :       4018 lignes
   ✅ OK DIM_GEOGRAPHIE                           :       3788 lignes
   ✅ OK DIM_ACTIVITE_NAF                         :         18 lignes
   ✅ OK DIM_ENTREPRISE                           :      19028 lignes
   ✅ OK DIM_BORNE_RECHARGE                       :        913 lignes
   ✅ OK DIM_USER_API                             :          4 lignes
   ✅ OK DIM_VEHICULE                             :      63322 lignes
   ✅ OK DIM_PRODUIT_PNEU                         :      11442 lignes
   ✅ OK DIM_CARACTERISTIQUES_PNEU                :        687 lignes
   ✅ OK DIM_DIMENSIONS_PNEU                      :        747 lignes
   ✅ OK FAIT_PRIX_PNEU                           :      11180 lignes
   ✅ OK FAIT_DISPONIBILITE_BORNE                 :        685 lignes
   ✅ OK FAIT_ENTREPRISE_AUTOMOBILE               :      12398 lignes

📊 SY

📦 Cellule 10 : Génération du Rapport Final


In [22]:
print("\n" + "=" * 70)
print("📊 GÉNÉRATION DU RAPPORT FINAL DE TESTS")
print("=" * 70)

# ========================================
# CRÉER LE DATAFRAME DU RAPPORT
# ========================================
df_rapport = pd.DataFrame(rapport_tests)

print(f"\n✅ Rapport généré : {len(df_rapport)} tests effectués")

# ========================================
# STATISTIQUES GLOBALES
# ========================================
nb_ok = len(df_rapport[df_rapport['Statut'] == '✅ OK'])
nb_ko = len(df_rapport[df_rapport['Statut'] == '❌ KO'])
nb_warning = len(df_rapport[df_rapport['Statut'] == '⚠️  WARNING'])
nb_erreur = len(df_rapport[df_rapport['Statut'] == '❌ ERREUR'])

print(f"\n📈 STATISTIQUES DES TESTS :")
print(f"   • Tests réussis (OK)       : {nb_ok} ({nb_ok/len(df_rapport)*100:.1f}%)")
print(f"   • Tests échoués (KO)       : {nb_ko} ({nb_ko/len(df_rapport)*100:.1f}%)")
print(f"   • Tests avec warning       : {nb_warning} ({nb_warning/len(df_rapport)*100:.1f}%)")
print(f"   • Tests en erreur          : {nb_erreur} ({nb_erreur/len(df_rapport)*100:.1f}%)")

# ========================================
# AFFICHER LE RAPPORT COMPLET
# ========================================
print("\n" + "=" * 70)
print("📋 RAPPORT DÉTAILLÉ DES TESTS")
print("=" * 70)
print(df_rapport.to_string(index=False))

# ========================================
# SAUVEGARDER LE RAPPORT
# ========================================
date_rapport = datetime.now().strftime('%Y%m%d_%H%M%S')
fichier_rapport = f"rapport_qualite_donnees_{date_rapport}.csv"

try:
    df_rapport.to_csv(fichier_rapport, index=False, encoding='utf-8-sig', sep=';')
    print(f"\n✅ Rapport sauvegardé : {fichier_rapport}")
except Exception as e:
    print(f"\n⚠️  Erreur lors de la sauvegarde : {e}")

# ========================================
# AFFICHER LES TESTS ÉCHOUÉS
# ========================================
if nb_ko + nb_erreur > 0:
    print("\n" + "=" * 70)
    print("⚠️  TESTS ÉCHOUÉS ET ERREURS")
    print("=" * 70)
    
    df_echecs = df_rapport[df_rapport['Statut'].isin(['❌ KO', '❌ ERREUR'])]
    print(df_echecs[['N°', 'Test', 'Table', 'Statut', 'Commentaire']].to_string(index=False))
    
    print("\n💡 ACTIONS CORRECTIVES RECOMMANDÉES :")
    print("   1. Analyser chaque test échoué individuellement")
    print("   2. Vérifier les données sources")
    print("   3. Corriger les ETL si nécessaire")
    print("   4. Ré-exécuter les tests après correction")

# ========================================
# CONCLUSION
# ========================================
print("\n" + "=" * 70)
print("🎯 CONCLUSION")
print("=" * 70)

taux_reussite = (nb_ok / len(df_rapport)) * 100

if taux_reussite >= 95:
    print(f"✅ QUALITÉ EXCELLENTE : {taux_reussite:.1f}% de tests réussis")
    print("   → Le Data Warehouse est prêt pour la production")
elif taux_reussite >= 80:
    print(f"⚠️  QUALITÉ ACCEPTABLE : {taux_reussite:.1f}% de tests réussis")
    print("   → Quelques corrections mineures recommandées")
else:
    print(f"❌ QUALITÉ INSUFFISANTE : {taux_reussite:.1f}% de tests réussis")
    print("   → Corrections majeures nécessaires avant mise en production")

print("\n" + "=" * 70)
print("📄 LIVRABLES GÉNÉRÉS :")
print(f"   • Rapport CSV : {fichier_rapport}")
print(f"   • Notebook exécuté : 8_data_qualite__.ipynb")
print("=" * 70)


📊 GÉNÉRATION DU RAPPORT FINAL DE TESTS

✅ Rapport généré : 59 tests effectués

📈 STATISTIQUES DES TESTS :
   • Tests réussis (OK)       : 58 (98.3%)
   • Tests échoués (KO)       : 1 (1.7%)
   • Tests avec warning       : 0 (0.0%)
   • Tests en erreur          : 0 (0.0%)

📋 RAPPORT DÉTAILLÉ DES TESTS
 N°                                  Test                                         Table                             Objectif                    Résultat   Seuil Statut            Commentaire
  1                   Nullité PK SK_Temps                                     DIM_TEMPS          PK ne doit jamais être NULL                           0       0   ✅ OK                       
  2              Nullité PK SK_Geographie                                DIM_GEOGRAPHIE          PK ne doit jamais être NULL                           0       0   ✅ OK                       
  3                Nullité PK SK_Activite                              DIM_ACTIVITE_NAF          PK ne doit jamais être NULL

📦 Cellule 11 : Fermeture des Connexions


In [23]:
print("\n" + "=" * 70)
print("🔒 FERMETURE DES CONNEXIONS")
print("=" * 70)

# ========================================
# FERMETURE DU CURSEUR DWH
# ========================================
try:
    if 'cursor_dwh' in locals() and cursor_dwh:
        cursor_dwh.close()
        print("✅ Curseur DWH fermé")
except Exception as e:
    print(f"⚠️  Erreur lors de la fermeture du curseur : {e}")

# ========================================
# FERMETURE DE LA CONNEXION DWH
# ========================================
try:
    if 'cnxn_dwh' in locals() and cnxn_dwh:
        cnxn_dwh.close()
        print("✅ Connexion DWH fermée")
except Exception as e:
    print(f"⚠️  Erreur lors de la fermeture de la connexion : {e}")

print("\n" + "=" * 70)
print("✅ PHASE 5 TERMINÉE - TESTS ET VALIDATION")
print("=" * 70)
print("\n🎓 COMPÉTENCE C15 VALIDÉE :")
print("   ✅ Qualité des données garantie")
print("   ✅ Intégrité référentielle vérifiée")
print("   ✅ Formatage des données conforme")
print("   ✅ Tests documentés et traçables")
print("\n🎉 FIN DU NOTEBOOK - RAPPORT DE QUALITÉ GÉNÉRÉ")
print("=" * 70)


🔒 FERMETURE DES CONNEXIONS
✅ Curseur DWH fermé
✅ Connexion DWH fermée

✅ PHASE 5 TERMINÉE - TESTS ET VALIDATION

🎓 COMPÉTENCE C15 VALIDÉE :
   ✅ Qualité des données garantie
   ✅ Intégrité référentielle vérifiée
   ✅ Formatage des données conforme
   ✅ Tests documentés et traçables

🎉 FIN DU NOTEBOOK - RAPPORT DE QUALITÉ GÉNÉRÉ


✅ RÉCAPITULATIF DU NOTEBOOK
Ton notebook 8_data_qualite__.ipynb contient maintenant :

✅ Cellule 1 : Imports et configuration
✅ Cellule 2 : Connexion au DWH
✅ Cellule 3 : Initialisation du rapport
✅ Cellule 4 : Test 1 - Nullité PK/FK
✅ Cellule 5 : Test 2 - Unicité PK
✅ Cellule 6 : Test 3 - Intégrité référentielle
✅ Cellule 7 : Test 4 - Doublons clés métier
✅ Cellule 8 : Test 5 - Format et cohérence
✅ Cellule 9 : Test 6 - Volumétrie
✅ Cellule 10 : Génération du rapport final
✅ Cellule 11 : Fermeture des connexions
📄 Livrables produits :

Rapport CSV horodaté
Affichage console complet
Statistiques de qualité
🎓 Compétence C15 validée ! 🚀